# JRC Global Surface Water 


- **Special requirements:** A Google account, access to Google Earth Engine.

- **Prerequisites:** You should have completed the "Week 5 - Prac 1" and "Week 5 - Prac 2" notebooks.

## Description
The Joint Research Centre's Global Surface Water (GSW) product (GEE data catalog page is [here](https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_4_GlobalSurfaceWater)) contains maps of the location and temporal distribution of surface water from 1984 to 2022 and provides statistics on the extent and change of those water surfaces. It is built using the Landsat satellite suite. This practical will explore two uses of this dataset. Firstly, we create a binary map of where permanent water bodies exist in the landscape, and secondly we extract a monthly time-series of water extent for a Lake in Botswana.

For more information see the associated journal article: [High-resolution mapping of global surface water and its long-term changes](https://www.nature.com/articles/nature20584.epdf?author_access_token=C5JSvooRop4jWxyp_qRPLNRgN0jAjWel9jnR3ZoTv0MqBuzCNsmw_DFxRd7sX93nfPzcbm_xTiPLlZMl7XrUhadm6EiT9cGdDNgn1s6EWrPWH3IeadLUjApplBoaS6xH).

## Aims of the practical session
* Load and display the GSW dataset over NSW
* Threshold the `occurence` layer to make a water/no-water binary mask
* Calcuate a time-series of water extent over a Lake in Botswana using the `JRC Monthly Water History` layer

## Load packages

Import GEE packages that are needed for the analysis.

In [ ]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
# ee.Authenticate()

## Connect to Google Earth Engine (GEE)

Connect to the GEE to have access computing tools and GEE datasets.
You may be required to input your Google account for authorization.

In [ ]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayerControl()

## Adding Region of Interest (ROI)

Create ROI that we want to work on it and then add and display it on the GEE map. In this example, we'll import a geojson from github that marks the boundary of NSW region.

In [ ]:
# load NSW geojson
path = 'https://raw.githubusercontent.com/nicolasyounes/engn3903/main/figures/NSW.geojson'
geometry = geemap.geojson_to_ee(path)

## Load and plot the GSW Occurrence layer

The Water `Occurrence` layer shows the frequency with which surface water occurred between 1984 and 2022. This product captures both the intra and inter-annual variability and changes. Values range from 0 % (water never occurs here), to 100% where water always occurs.

Water is a scarce feature in the landscape so zoom in to get a better look at the dataset.

In [ ]:
# load latest version of GSW
gsw = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')

# To display GSW Occurance layer
Map.addLayer(gsw.clip(geometry),
             {'bands': ['occurrence'],
             'palette': ['red', 'blue'],
             'max':100,
             'min':0
             },
             'Water Occurrence (1984-2022')

Map.centerObject(geometry)
Map

## Create a map of permanent waterbodies

Quite often in remote sensing we require maps that show us where permanent water bodies are. We often use these to mask other datasets (among many other uses). For example, when creating maps of vegetation condition, its often helpful to mask out water bodies. Below we threshold the `occurrence` layer to define pixels that are permanently water (or almost always water). Feel free to experiment with different thresholds.  It may help to zoom into a specific water body to see the impacts.

In [ ]:
#pixels whose value is greater than the threshold will be returned as 1.
threshold=90 
waterbodies = gsw.select('occurrence').gt(threshold)

# To display GSW Occurance layer
Map.addLayer(waterbodies.clip(geometry),
             {
             'palette': ['white', 'black']
             },
             '90% occurrence water mask')
Map

<div class="alert alert-block alert-danger"> 

## Exercise 1

- For all of Australia, create a map of ephermeral water bodies. Define ephemeral as pixels where the water occurence value is less than 30 %. You can use the [Large Scale International Boundary (LSIB)](https://developers.google.com/earth-engine/datasets/catalog/USDOS_LSIB_SIMPLE_2017) dataset to define the Australian extent. If all of Australia takes too long to load, use another state boundary instead

    
</div>    

## Monthly time series of water extent in Lake Ngami

The [JRC Monthly Water History](https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_4_MonthlyHistory?hl=en) dataset provides information on all the water detections at the monthly level from March 1984 to Jan 2022. It can be used to extract a time-series of water extent for a given waterbody.  In the code below we will learn how to produce a time-series of water-body extent for a Lake in Botswana

### Load JRC Monthly Water History

Using a polygon that outlines Lake Ngami in Botswana, we will load the monthly history of water extent from 2013 to 2021.

In [ ]:
# load the geojson
path =  'https://raw.githubusercontent.com/nicolasyounes/engn3903/main/figures/Lake_Ngami.geojson'
geometry = geemap.geojson_to_ee(path)

#load the jrc dataset, clip tp geometry and a time-range
water_ts = ee.ImageCollection('JRC/GSW1_4/MonthlyHistory').filterDate('2013-06-01', '2021-12-31').map(lambda image: image.clip(geometry))

### Reclassify the datasets

Pixels in this dataset have one of three possible values:

0 = No Observations, 1 = Not-Water, 2 = Water

We will reclassify this dataset into a binary water/no-water dataset to make calculating area more simple. We can do that by simply thresholding the dataset

In [ ]:
#convert the water history dataset into a binary image 1=water
water = water_ts.map(lambda image: image.gt(1))

### Plot the datasets

Along with plotting the first image in the ImageCollection we just loaded, we will also plot a time-series of annual Landsat composites to give us an impression of how the Lake's extent is changing through time. This can give us a visualise expectation for the final graph of water-area through time.

In [ ]:
# Create a new map object
Map1 = geemap.Map()
Map1.add_basemap('SATELLITE')
Map1.addLayerControl()
Map1.centerObject(geometry)

# add the first water extent map to the map
Map1.addLayer(water_ts.first().selfMask(), 
             {'bands': ['water'],
              'min': 0.0,
              'max': 2.0,
              'palette': ['ffffff', 'fffcb8', '0905ff']

             },
             'Monthly Water Extent (first image)'
             )

# Add the reclassified water etxet to the map
Map1.addLayer(water.first(), 
             {'bands': ['water'],
              'min': 0.0,
              'max': 1.0,
              'palette': ['ffffff', 'black']

             },
             'Reclassified Water Extent (first image)'
            )

#add a GIF of Landsat to visualize how the lake's water extent is changing
Map1.add_landsat_ts_gif(
    roi=geometry,
    label='Lake Ngami',
    start_year=2013,
    end_year=2021,
    start_date='01-01',
    end_date='12-31',
    bands=['SWIR1', 'NIR', 'Red'],
    font_color='white',
    frames_per_second=3,
    progress_bar_color='blue',
)

Map1

### Calculate Water Area through time

In [ ]:
#Function for converting the binary raster into vectors and calculating area of water 
def ras_to_vec(img):
    vec = img.selfMask().reduceToVectors(geometry=geometry, scale=30, maxPixels=1e13)
    vec = vec.filterBounds(geometry)
    return vec.set({'area': vec.geometry().area(1).divide(1e6).round()})

#apply the function to the image collection
water_vectors = water.map(ras_to_vec)
areas = water_vectors.aggregate_array('area')

#Grab the dates from the imagecollection
dates = water_ts.aggregate_array('system:time_start').map(
    lambda d: ee.Date(d).format('YYYY-MM-dd')
)

# Add to Pandas dataframe
x = dates.getInfo()
y = areas.getInfo()
df = pd.DataFrame({'Date':x,  'Area':y}).set_index('Date')

### Plot water extent through time

In [ ]:
df.plot(figsize=(15, 5))
plt.ylabel('Water Area (km2)')

<div class="alert alert-block alert-danger"> 

## Exercise 2

- Describe the water extent plot above. Can you hypothesize why there might be a trend? What other datasets might help elucidate the reasons for the trend you see? You may find it helpful to rerun the analysis over a longer time period (e.g. 2000-2020).

    
</div>    

## References
This is where the references go. For exmaple:

* Pekel, J.F., Cottam, A., Gorelick, N. and Belward, A.S., 2016. High-resolution mapping of global surface water and its long-term changes. Nature, 540(7633), pp.418-422.

* Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 

**Contact:** If you need assistance, please post a question on the ENGN3903 Wattle (**check**) site 

**Last modified:** Sept. 2023